# 12 다중 머신과 장치를 위한 분산 텐서플로



## 12.1 단일 머신의 다중 장치

텐서플로우의 분산 컴퓨팅 기능은 다른 신경망 프레임워크와 비교되는 주요 장점이다.  
한 대의 머신으로 간단한게 성능을 올릴 수 있는 방법은 gpU카드를 추가하는 것이다. 대부분의 경우에는 이로 충분하다.  
다시말하면, 여러 대의 컴퓨터로 나뉜 16개의 gpu보다 한대의 컴퓨터에 설치된 8개의 gpu가 더 빠를 수 있다.(네트워크 통신 지연 때문)   
NVIDIA의 CUDA(Compute Unified Device Architecture)를 보통 이용.

Colab 도구 -> 런타임 유형 변경 -> GPU 사용 체크 하면 !nvidia-smi 명령어에 다움과 같이 CUDA 드라이버 버전 정보 등이 나온다

In [0]:
!nvidia-smi

In [0]:
# !pip install --upgrade tensorflow-gpu ##Colab은 이미 기본적으로 가지고있음 실행 X

### 12.1.2 GPU RAM 관리  

  기본적으로 텐서플로는 계산 그래프가 처음 실행될 때 가용한 모든 GPU의 RAM을 자동으로 확보한다.  
  GPU 메모리의 40%만 잡게하려면 ConfigProto 객체를 만들어서 gpu_options.per_process_gpu_memory_fraction 옵션을 0.4로 지정하고 이 설정으로 세션을 생성.  

Tensorflow2는 ConfigProto()를 지원하지 않음 -> compat.v1.ConfigProto

In [0]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config = config)

### 12.1.3 장치에 연산 배치하기
동적 배치자: 가용한 모든 장치에 연산을 완전히 자동으로 분산하는 알고리즘  (아직은 구글 내부용 공개 X)  
텐서플로우는 __단순 배치자__ 에 의존하고 있다.

#### 단순 배치
- 이전 그래프 실행에서 어떤 노드가 이미 한 장치에 배치되어 있다면 그 노드는 그 장치에 그대로 둔다.  
- 만일 사용자가 노드를 어떤 장치에 할당했다면 배치자는 노드를 그 장치에 배치한다.  
- 그 외에는 GPU #0이 기본으로 사용되고 GPU가 없으면 CPU가 사용된다.

In [0]:
with tf.device("/cpu:0"):
  a = tf.Variable(3.0)
  b = tf.constant(4.0)

c = a * b

In [0]:
c.eval

#### 배치 로깅
단순 배치자가 우리가 지정한 배치를 따르는지 Check

In [0]:
config = tf.compat.v1.ConfigProto()
config.log_device_placement = True
config

In [0]:
def variables_on_cpu(op):
  if op.type == "Variable":
    return "/cpu:0"
  else:
    return "/gpu:0"

with tf.device(variables_on_cpu):
  a = tf.Variable(3.0)
  b = tf.constant(4.0)
  c = a * b

#### 연산과 커널
많은 연산이 CPU와 GPU를 위한 커널을 가지고 있지만 전부는 아니다.  
예를들면, 텐서플로는 정수 변수에 대한 GPU커널은 가지고 있지 않다..

#### 간접 배치
기본적으로 커널이 없는 장치에 연산을 할당하면 텐서플로가 그 장치에 연산을 배치할떄 앞서 본 것처럼 에러를 발생. 에러를 발생시키는 대신 텐서플로가 CPu를 사용하도록 하려면 allow_soft_placement 환결 설정을 True로 지정하면된다

### 12.1.4 병렬 실행
CPU의 평가 큐에 있는 연산은__inter-op 스레드 풀__ 로 보내진다. CPu가 여러개의 코어를 사용한다면 이 연산들은 병렬화 되어 효율적으로 평가 될 것이다. 일부 연산은 멀티스레드 CPU 커널을 가지고 있다. 이런 커널은 작업을 열거해 부분연산으로 쪼개어 다른 평가 큐에 배치하고 __intra-op 스레드 풀__ 로 보내진다. 요약하면 어러개의 연산과 부분연산이 다른 CPU 코어에서 병렬로 평가될 수 있다.

GPU의 경우는 GPU의 평가 큐에 있는 연산은 그냥 순서대로 평가된다. 그러나 많은 연산이 텐서플로가 사용하는 CUDA와 cuDNN 같은 라이브러리로 구현된 멀티스레드 GPU 커널을 가지고있다.

### 12.1.5 제어 의존성
어떠한 경우에는 의존하는 모든 연산이 실행되었음에도 불구하고 연산의 평가를 미루는 것이 나을 때도 있다. 예를들어 메모리를 많이 사용하는 연산이지만 그 값이 그래프의 후반에서야 필요하다면 다른 연산에 필요한 메모리를 불필요하게 점유하는 것을 피하기 위해 마지막 순간에 평가하는 것이 최선이다. 일부 노드의 평가를 지연시키는 간단한 방법은 __제어 의존성__ 을 추가하는 것이다

In [0]:
a = tf.constant(1.0)
b = a+ 2.0

with tf.control_dependencies([a,b]):
  x = tf.constant(3.0)
  y = tf.constant(4.0)

z = x + y 

지금까지의 내용을 요약하면  
1. 연산을 여러 장치에 원하는 방식으로 배치하는 방법
2. 이런 연산들을 병렬로 실행하는 방법
3. 병렬 실행을 최적화하기 위해 제어 의존성을 만드는 방법

## 12.2 다중머신의 다중 장치  
여러대의 머신에서 그래프를 실행하려면 먼저 클러스터를 정의해야 한다.  
__클러스터__ 는 __태스크__ 라고하는 하나 이상의 텐서플로 서버로 구성되며 보통 여러 대의 머신에 나뉘어 있습니다. 각 태스크는 하나의 잡에 속해 있습니다.  
다음의 클러스터 명세는 각각 한개와 두개의 태스크를 가지는 잡 ps와 worker를 정의

In [0]:
cluster_spec = tf.train.ClusterSpec({
    "ps" : ["machine-a.example.com:2221",
            ],
    "worker": ["machine-a.example.com:2222", 
               "machine-b.example.com:2222",
               ]})

### 12.2.1 세션 열기


모든 태스크가 시작되면(실제로는 아무것도 실행되지않음) 어떤 머신의 프로세스에 잇는 클라이언트에서도 다른 모든 서버에 대해 세션을 열수 있다.

```python
a = tf.constant(1.0)
b = a + 2
c = a * 3

with tf.Session("grpc:///machine-b.example.com:2222") as sess:
    print(c.eval()) #9.0
```

### 12.2.2 마스터와 워커서비스


클라이언트는 gRPC 프로토콜(구글 원격 프로시저 호출)을 사용하여 서버와 통신 데이터는 구글의 또다른 오픈소스 기술인 프로토콜 버퍼 형태뢔 전달된다  
모든 텐서플로 서버는 __마스터 서비스__ 와 __워커 서비스__ 두개의 서비스를 제공한다.  
마스터 서비스는 클라이언트가 세션을 열고 그래프를 실핼항 수 있게 도와준다.  여러 태스크의 계산을 조율하고 워커 서비스를 통해 실제로 로컬 장치에서 계산을 실행하고 결과를 받는다.

### 12.2.3 여러태스크에 연산 할당하기

장치 블록을 사용하여 어떤 태스크에서 관리하는 장치에  
`잡 이름`, `태스크 번호`, `장치 유형`, `장치 번호` 를 지정하여 연산을 할당 할 수 있다.
```python
with tf.device('/job:ps/task:0/cpu:0'):
    a = tf.constant(1.0)

with tf.devcie("/job:worker/task:0/gpu:1'):
    b= a + 2
            
c = a + b 
```

### 12.2.4 여러대의 파라미터 서버에 변수를 나누어 분산하기 

분산 환경에서 신경망을 훈련시킬 떄 일반적인 패턴은 모델 파라미터를 일련의 파라미터 서버에 저장하는 것이다.  
반면 다른 태스크는 연산에 집중한다. 수백만개의 파라미터가 있는 대규모 모델의 경우에는 파라미터 서버 한대의 네트워크 카드가 포화되는 것을 피하기 위해 파리미터들을 여러 대의 파라미터 서버에 나누는것이 좋다.  
텐서플로는 라운드 로빈 방식으로 변수들을 분산시켜주는 replica_device_setter() 함수를 제공한다.

```python
with tf.device(tf.train.replica_device_setter(ps_tasks=2):
     v1 = tf.Variable(1.0) # /job:ps/task:0에 할당
     v2 = tf.Variable(2.0) # /job:ps/task:1에 할당
     v3 = tf.Variable(3.0) # /job:ps/task:0에 할당
     v4 = tf.Variable(4.0) # /job:ps/task:1에 할당
     v5 = tf.Variable(5.0) # /job:ps/task:0에 할당

```

### 12.2.5 리소스 컨테이너를 사용해 여러 세션에서 상태 공유하기

분산 환경이 아닌 평범한 __로컬 세션__을 사용할 때 각 변수의 상태는 세션 자체에서 관리한다. 세션이 종료되면 모든 변수가 사라진다. 반대로 분산 세션을 사용할 때는 변수의 상태가 세션이 아니라 클러스터 자체에 있는 리스소 컨테이너에 의해 관리된다.

### 12.2.6 텐서플로 큐를 사용한 비동기 통신

큐는 여러 세션 사이에 데이터를 교환하기 위한 아주 좋은 또 다른 방법이다. 예를 들면 한 클라이언트느 훈련 데이터를 로드하여 큐에 저장하는 그래프를 만들고, 다른 클라이언트는 큐에서 데이터를 추출하여 모델을 훈련시키는 그래프를 만드는 것이다. 이렇게하면 훈련 연산이 매스텝마다 다음번 미니배치를 기다리지 않아도 되기때문에 훈련 속도가 빨라진다.  
가장 대표적인 큐방법은 FIFO(first in first out, 선입선출)

```python
 q = tf.FiFoQueue(capacity = 10, dtypes = [tf.float32], shapes = [[2]], name = "q", shared_name = "shared_q")
```

#### 큐에 데이터 넣기

큐에 데이터를 넣기 위해선 enqueue 연산을 만들어야한다.
```python
training_instance = tf.placeholder(tf.float32, shape(2))
enqueue = q.enqueue([training_instance])

with tf.Session("grpc://machine-a.example.com:2222") as sess:
    sess.run(enqueue, feed_dict={training_instance: [1., 2.]})
    sess.run(enqueue, feed_dict={training_instance: [3.,4.]})
    sess.run(enqueue, feed_dict={training_instance: [5., 6.]})
    ```
    
enqueue_many 를 사용하면 한번에 여러개를 넣을 수 있다.

#### 큐에서 데이터 추출하기

큐에서 데이터를 추출하기위해선 dequeue를 사용..

#### 튜플 큐 
큐에 있는 각 아이템은 한아ㅢ 텐서 대신 텐서의 튜플이 될 수 있다.

####   큐 종료
다른 세션에 시그널을 보내 더 이상 데이터가 추가 되지 않을 큐를 종료하는 것이 가능하다

#### RandomShuffleQueue
FIFO와 같이 다양한 종류의 큐타입을 제공한다. RandomShuffleQueue는 그일종. 아이템은 랜덤하게 변환하는 큐타입이다.

#### PaddingFifoQueue
PaddingFifoQueue는 어떤 차원으로든지 크기가 다른 텐서를 받을 수 있다는것을 제외하고는 FiFOQueue와 동일하다.

### 12.2.7 그래프에서 직접 데이터 로드하기

지금까지의 방법들은 훈련데이터를 로드하고 플레이스홀더를 사용해 클러스터에 데이터를 주입한다고 가정했다. 간단하고 작은 규모의 환경에서는 잘작동하지만 훈련데이터를 다음과같이 여러번 전송하기 때문에 비효율적임
1. 파일 시스템에서 클라이너트로
2. 클라이언트에서 마스터 태스크로
3. 마스터태스크에서 이 데이터가 필요한 다른 태스크로

#### 데이터를 변수에 프리로드하기
데이터셋이 메모리 크기에 맞는다면 훈련 데이터를 한번에 로드해서 변수에 할당하고 그래프에서 이 변수를 바로 사용하는것이 더 좋은 방법이다. 이를 훈련 세트를 __프리로딩__ 한다고한다.

#### 그래프에서 직접 훈련 데이터 읽기
훈련 데이터가 메모리 크기에 맞지 않으면 __리더 연산__을 사용하는 것이 좋다.
- CSV
- 고정 길이의 이진 레코드
- 프로토콜 버퍼 기반의 텐서플로의 TFRecords

먼저 파일을 읽기 위한 TextLineReader를 만든다.  
그다음 어떤 파일을 읽어야 할지 리더에 알려주기 위해 큐를 만든다.  
enqueue 연산을 만들고 원하는 파일 이름을 큐에 넣기 위해 플레이스 홀더를 만든다.  큐를 종료하기위해 종료연산도 만든다.

## 12.3 텐서플로 클러스터에서 신경망 병렬화하기

### 12.3.1 장치마다 하나의 신경망

텐서플로 클러스터에서 신경망을 훈련하고 실행시키는 가장 간단한 방법은 하나의 머신에 있는 하나의 장치를 사용하는 것과 똑같은 식으로 코드를 작성하는것이다.  그러고 나서 세션을 만들 때 마스터 서버의 주소를 지정한다.  
__초당 쿼리(QPS)__를 받아 신경망이 각 쿼리에 대한 예측을 수행하는 웹 서비스를 제공하는 경우에도 좋다.

### 12.3.2 그래프 내 복제와 그래프 간 복제

여러 신경망을 다른 장치에 배치하여 신경망의 대규모 앙상블 훈련을 쉽게 병렬화할 수도 있다. 그러나 앙상블을 실행할 때는 각 신경망이 만든 개개의 예측을 모아서 앙상블의 예측을 만들어야 한다. 이렇게 하려면 약간의 조율이 필요하다.

- 각각 다른 장치에 할당된 모든 신경망을 담는 하나의 큰 그래프를 만든다. 모든 신경망으로부터 각가의 예측을 모으는 계산도 포함된다. 그다음에 클러스터에 있는 한 서버에 세션을 만들고 모든 계산을 위임한다. 이런방식을 __그래프 내 복제__ 라고한다
- 각 신경망을 독립된 그래프로 만들고 이 그래프 사이의 동기화를 직접 관리할 수 있습니다. 이런 방식을 그래프 간 복제 라고한다. 전형적으로는 큐를 사용하여 그래프를 조율한다.

### 12.3.3 모델 병렬화

지금까지는 하나의 장치에 하나의 신경망을 실행했다 그러면 하나의 신경망으로 여러개의 장치에서 실행하려면 어떻게 해야할까?  
그렇게 하려면 모델을 여러 부분으로 나누어 각 부분을 다른 장치에서 실행 시켜야한다. 이를 __모델 병렬화__ 라고한다. 모델병렬화는 매웅렵고 신경말 모델 구조에 매우 의존적이라 FCNN인 경우에 별로 이득이 없다. 하지만 CNN의 경우에는 이전 층에 부분적으로만 연결된 층을 가지기 때문에 분산 시키기가 훨씬 쉽다. 

### 12.3.4 데이터 병렬화

신경망의 훈련을병렬화하는 또 다른 방법은 각 장치에 모델을 복제해서 각각 다른 미니배치를 사용해 모든 모델이 동시에 훈련스텝을 실행하고 그래디언트를 취합하여 ㅂ모델 파라미터를 업데이트 하는것이다 이를 __데이터 병렬화__ 라고한다.
- 동기 업데이트 
동기업데이트에서는 그래디언트 수집기가 모든 그래디언트가 계산될 떄까지 기다려서평균을 계산하고 결과를 반영한다.
- 비동기 업데이트
비동기 업데이트에서는 복제 모델이 그래디언트 계산을 끝낼 떄 마다 즉시 이를 사용해 모델 파라미터를 업데이터한다 여기에는 취합 단계가 없고 동기화도 없다 복제 모델들은 다른 복제 모델과 독립적으로 작동한다. 다른 복제 모델을 대기하지 않기 때문에 이방식은 분당 더많은 훈련 스텝을 실행할 수 있다.